# Imports

In [32]:
import geojson as gj
import matplotlib.pyplot as plt
import folium as fl
import geopandas as gpd
import json
import pandas as pd
import networkx as nx
import pickle
import haversine as hs

In [33]:
# Load the file "/Users/johanmasvie/programming/tiø4900/master_thesis/Scigrid_data/IGGIN_Nodes.geojson" into a GeoDataFrame
nodes_gdf = gpd.read_file('/Users/johanmasvie/programming/tiø4900/master_thesis/Scigrid_data/IGGIN_Nodes.geojson')

nodes_gdf.head()

,name,id,country_code,tags,param,method,geometry
0,N_0,NO_N_0,NO,{},"{'eic_code': None, 'elevation_m': None, 'exact...","{'eic_code': None, 'elevation_m': None, 'exact...",POINT (6.79720 65.10640)
1,N_1,NO_N_1,NO,{},"{'eic_code': None, 'elevation_m': None, 'exact...","{'eic_code': None, 'elevation_m': None, 'exact...",POINT (5.19357 59.33286)
2,N_2,NO_N_2,NO,{},"{'eic_code': None, 'elevation_m': None, 'exact...","{'eic_code': None, 'elevation_m': None, 'exact...",POINT (2.89575 60.56023)
3,N_3,NO_N_3,NO,{},"{'eic_code': None, 'elevation_m': None, 'exact...","{'eic_code': None, 'elevation_m': None, 'exact...",POINT (2.89614 60.78219)
4,N_4,NO_N_4,NO,{},"{'eic_code': None, 'elevation_m': None, 'exact...","{'eic_code': None, 'elevation_m': None, 'exact...",POINT (3.29837 54.78877)


In [34]:
# Open the file: "/Users/johanmasvie/programming/tiø4900/master_thesis/Scigrid_data/IGGIN_PipeSegments.csv" as a GeoDataFrame: gdf
pipelines_gdf = gpd.read_file('/Users/johanmasvie/programming/tiø4900/master_thesis/Scigrid_data/IGGIN_PipeSegments.csv')

# Split the 'param' column into separate columns
pipelines_gdf['param'] = pipelines_gdf['param'].apply(lambda x: json.loads(x.replace("'", '"')))
param_df = pd.json_normalize(pipelines_gdf['param'])
pipelines_gdf = pipelines_gdf.drop(columns=['param']).join(param_df)

# Split the 'uncertainty' column into separate columns and add them to the GeoDataFrame, prefaced by the string 'uncertainty_'
pipelines_gdf['uncertainty'] = pipelines_gdf['uncertainty'].apply(lambda x: json.loads(x.replace("'", '"')))
uncertainty_df = pd.json_normalize(pipelines_gdf['uncertainty'])
uncertainty_df.columns = ['uncertainty_' + col for col in uncertainty_df.columns]
pipelines_gdf = pipelines_gdf.drop(columns=['uncertainty']).join(uncertainty_df)

# Split the 'method' column into separate columns and add them to the GeoDataFrame, prefaced by the string 'method_'
pipelines_gdf['method'] = pipelines_gdf['method'].apply(lambda x: json.loads(x.replace("'", '"')))
method_df = pd.json_normalize(pipelines_gdf['method'])
method_df.columns = ['method_' + col for col in method_df.columns]
pipelines_gdf = pipelines_gdf.drop(columns=['method']).join(method_df)

# Split the "country_code" column into two separate columns. "['ES', 'ES']" is an example of a value in the "country_code" column
pipelines_gdf['country_code'] = pipelines_gdf['country_code'].apply(lambda x: x.replace('[', '').replace(']', '').replace("'", '').split(', '))
country_code_df = pd.DataFrame(pipelines_gdf['country_code'].to_list(), columns=['country_code_1', 'country_code_2'])
pipelines_gdf = pipelines_gdf.drop(columns=['country_code']).join(country_code_df)

# Split the node_id column into two separate columns. "['INET_N_23', 'INET_N_295']" is an example of a value in the "node_id" column
pipelines_gdf['node_id'] = pipelines_gdf['node_id'].apply(lambda x: x.replace('[', '').replace(']', '').replace("'", '').split(', '))
node_id_df = pd.DataFrame(pipelines_gdf['node_id'].to_list(), columns=['node_id_1', 'node_id_2'])
pipelines_gdf = pipelines_gdf.drop(columns=['node_id']).join(node_id_df)

# Print the first few rows of the GeoDataFrame
pipelines_gdf.head()

,id,name,source_id,lat,long,comment,tags,geometry,diameter_mm,end_year,...,method_long_mean,method_max_cap_M_m3_per_d,method_max_pressure_bar,method_num_compressor,method_start_year,method_waterDepth_m,country_code_1,country_code_2,node_id_1,node_id_2
0,INET_PL_1,AlcazarDeSanJuan_Chinchilla,['INET_PL_1'],"[39.3900748, 38.892435]","[-3.3500984, -1.719831]",None,{},None,923.792529,2050,...,make_Attrib(latlong_mean),Lasso,Median,Median,make_Attrib(const),Lasso,ES,ES,INET_N_23,INET_N_295
1,INET_PL_3,Tarancon_Madrid,['INET_PL_3'],"[40.01, 40.223]","[-3.207, -3.681]",None,{},None,923.792529,2050,...,make_Attrib(latlong_mean),Lasso,Median,Median,make_Attrib(const),Lasso,ES,ES,INET_N_1460,INET_N_904
2,INET_PL_5,Alessandria_Cortemaggiore,['INET_PL_5'],"[44.991076, 44.90008]","[9.920891, 8.60944]",None,{},None,914.400000,2050,...,make_Attrib(latlong_mean),Lasso,raw,Median,make_Attrib(const),Lasso,IT,IT,INET_N_317,INET_N_26
3,INET_PL_7,Almendralejo_Badajoz,['INET_PL_7'],"[38.711202, 38.87665]","[-6.382539, -6.9714]",None,{},None,923.792529,2050,...,make_Attrib(latlong_mean),Lasso,Median,Median,make_Attrib(const),Lasso,ES,ES,INET_N_36,INET_N_108
4,INET_PL_9,Almendralejo_Zamora,['INET_PL_9'],"[38.711202, 41.506]","[-6.382539, -5.644]",None,{},None,923.792529,2050,...,make_Attrib(latlong_mean),Lasso,Median,Median,make_Attrib(const),Lasso,ES,ES,INET_N_36,INET_N_1653


In [35]:
# Create a graph of the pipeline network
G = nx.Graph()

# Add the nodes to the graph
for i, row in nodes_gdf.iterrows():
    G.add_node(row['id'], pos=(row['geometry'].x, row['geometry'].y))

# Add the edges to the graph
for i, row in pipelines_gdf.iterrows():
    G.add_edge(row['node_id_1'], row['node_id_2'])

In [36]:
# Add a column to the nodes_gdf GeoDataFrame that contains a number symbolizing the connected component the node is a part of
nodes_gdf['connected_component'] = 0
for i, component in enumerate(nx.connected_components(G)):
    for node in component:
        nodes_gdf.loc[nodes_gdf['id'] == node, 'connected_component'] = i

In [37]:
# Save the graph object to a file
with open('pipeline_network.pkl', 'wb') as f:
    pickle.dump(G, f)

In [38]:
# Plot the nodes and pipeline segments in a folium map
m = fl.Map(location=[nodes_gdf['geometry'].y.mean(), nodes_gdf['geometry'].x.mean()], zoom_start=5)

# Create a list of colors for each connected component
colors = ['red', 'darkblue', 'darkgreen', 'gray', 'white', 'beige']

# Create a feature group for the node markers
node_markers = fl.FeatureGroup(name='Node Markers')

for i, row in nodes_gdf.iterrows():
    # Get the connected component of the node
    component = nodes_gdf.loc[nodes_gdf['id'] == row['id'], 'connected_component'].values[0]
    # Assign a color based on the connected component
    color = colors[component % len(colors)]
    fl.Marker([row['geometry'].y, row['geometry'].x], popup=row['id'], icon=fl.Icon(color=color)).add_to(node_markers)

# Add the node markers feature group to the map
node_markers.add_to(m)

# Create a feature group for the pipeline segments
pipeline_segments = fl.FeatureGroup(name='Pipeline Segments')

for i, row in pipelines_gdf.iterrows():
    # Get the connected component of the pipeline segment
    component = nodes_gdf.loc[nodes_gdf['id'] == row['node_id_1'], 'connected_component'].values[0]
    # Assign a color based on the connected component
    color = colors[component % len(colors)]
    fl.PolyLine([(nodes_gdf[nodes_gdf['id'] == row['node_id_1']]['geometry'].y.values[0], nodes_gdf[nodes_gdf['id'] == row['node_id_1']]['geometry'].x.values[0]),
                 (nodes_gdf[nodes_gdf['id'] == row['node_id_2']]['geometry'].y.values[0], nodes_gdf[nodes_gdf['id'] == row['node_id_2']]['geometry'].x.values[0])],
                color=color).add_to(pipeline_segments)

# Add the pipeline segments feature group to the map
pipeline_segments.add_to(m)

# Add the layer control to the map
fl.LayerControl().add_to(m)

m.save('pipeline_network.html')

In [39]:
# Save the map to a file
m.save('pipeline_network.html')

In [40]:
# Iterate over the nodes in the graph. If two graphs are within a certain distance of each other, but part of different connected components, do something
doubled_nodes = []

for i, node1 in nodes_gdf.iterrows():

    # Print the progress
    print(i, '/', len(nodes_gdf), end='\r')

    # Get the position of the first node and convert it to a tuple
    pos1 = (node1['geometry'].x, node1['geometry'].y)

    # Iterate over the nodes in the graph again
    for j, node2 in nodes_gdf.iterrows():

        # Get the position of the second node and convert it to a tuple
        pos2 = (node2['geometry'].x, node2['geometry'].y)

        # Calculate the Haversine distance between the two nodes
        distance = hs.haversine(pos1, pos2)

        # Check if the distance is less than 5 km and the nodes are part of different connected components
        if distance < 5 and node1['connected_component'] != node2['connected_component']:

            # Store the nodes in a list of tuples
            doubled_nodes.append((node1['id'], node2['id']))

In [41]:
# Using the list of tuples, and nodes_gdf, create a GeoDataFrame of the nodes that are within 5 km of each other, but part of different connected components
doubled_nodes_gdf = gpd.GeoDataFrame(doubled_nodes, columns=['node_id_1', 'node_id_2'])
doubled_nodes_gdf = doubled_nodes_gdf.merge(nodes_gdf, left_on='node_id_1', right_on='id')
doubled_nodes_gdf = doubled_nodes_gdf.merge(nodes_gdf, left_on='node_id_2', right_on='id', suffixes=('_1', '_2'))
doubled_nodes_gdf = doubled_nodes_gdf.drop(columns=['id_1', 'id_2'])

In [ ]:
# 